In [ ]:
import torch

class MaskTransformer(torch.nn.Module):
    def __init__(
        self,n_cls,patch_size,d_encoder,n_layers,n_heads,d_model,d_ff,drop_path_rate,dropout):
        super().__init__()
        self.d_encoder = d_encoder
        self.patch_size = patch_size
        self.n_layers = n_layers
        self.n_cls = n_cls
        self.d_model = d_model
        self.d_ff = d_ff

        self.cls_emb = torch.nn.Parameter(torch.randn(1, n_cls, d_model))

        self.proj_patch = torch.nn.Parameter(torch.randn(d_model, d_model))
        self.proj_classes = torch.nn.Parameter(torch.randn(d_model, d_model))

        self.decoder_norm = torch.nn.LayerNorm(d_model)
        self.mask_norm = torch.nn.LayerNorm(n_cls)

    def forward(self, x):
        cls_emb = self.cls_emb.expand(x.size(0), -1, -1)
        x = torch.cat([x, cls_emb], dim=1)
        x = self.transformer(x)

        patches, cls_seg_feat = x[:, : -self.n_cls], x[:, -self.n_cls :]

        patches = patches / patches.norm(dim=-1, keepdim=True)
        cls_seg_feat = cls_seg_feat / cls_seg_feat.norm(dim=-1, keepdim=True)